<a href="https://colab.research.google.com/github/bsewp045/CSI4900-Sarcasm-Detection/blob/main/train/oversample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
import pandas as pd

In [5]:
df = pd.read_csv('/content/drive/MyDrive/Dataset/train/train.En.csv')
df.head(5)

,Unnamed: 0,tweet,sarcastic,rephrase,sarcasm,irony,satire,understatement,overstatement,rhetorical_question
0,0,The only thing I got from college is a caffein...,1,"College is really difficult, expensive, tiring...",0.0,1.0,0.0,0.0,0.0,0.0
1,1,I love it when professors draw a big question ...,1,I do not like when professors don’t write out ...,1.0,0.0,0.0,0.0,0.0,0.0
2,2,Remember the hundred emails from companies whe...,1,"I, at the bare minimum, wish companies actuall...",0.0,1.0,0.0,0.0,0.0,0.0
3,3,Today my pop-pop told me I was not “forced” to...,1,"Today my pop-pop told me I was not ""forced"" to...",1.0,0.0,0.0,0.0,0.0,0.0
4,4,@VolphanCarol @littlewhitty @mysticalmanatee I...,1,I would say Ted Cruz is an asshole and doesn’t...,1.0,0.0,0.0,0.0,0.0,0.0


In [6]:
df[ (df['rhetorical_question'] == 1) & (df['sarcasm'] == 1) & (df['understatement'] == 1)][['sarcasm', 'irony', 'satire', 'understatement', 'overstatement', 'rhetorical_question']]

,sarcasm,irony,satire,understatement,overstatement,rhetorical_question
790,1.0,0.0,0.0,1.0,0.0,1.0


In [7]:
# Read training data
df = pd.read_csv('/content/drive/MyDrive/Dataset/train/train.En.csv')
df.drop(columns=df.columns[0], axis=1, inplace=True)
# Read testing data
df_test = pd.read_csv('/content/drive/MyDrive/Dataset/test/task_A_En_test.csv') 
# Read training sample which contains the sarcastic tweets and their rephrase
df_extracted = pd.read_csv('/content/drive/MyDrive/Dataset/train/sarcastic_tweets.csv')
df_extracted.drop(columns=df_extracted.columns[0], axis=1, inplace=True)
#Extract the sarcastic tweets only
df_sarcastic = df_extracted[df_extracted['sarcastic']==1]
df_not_sarcastic = df[df['sarcastic'] == 0][['tweet', 'sarcastic']]
rephrase = df_extracted[df_extracted['sarcastic'] == 0]
df_not_sarcastic = pd.concat([df_not_sarcastic,rephrase],axis=0, ignore_index=True )
df_not_sarcastic.head(5)

,tweet,sarcastic
0,I always think going braless is a good idea un...,0
1,life is so much better with a heating blanket,0
2,Sometimes I just go through my phone and look ...,0
3,was not back in the states for even 5 minutes ...,0
4,in desperate need of (and I can NOT stress thi...,0


In [8]:
count = df_not_sarcastic.shape[0]
# Oversample the sarcastic tweets
df_sarcastic_over = df_sarcastic.sample(count, replace=True)

df_balanced = pd.concat([df_not_sarcastic,df_sarcastic_over],axis=0, ignore_index=True)
df_balanced.shape

(6936, 2)

In [9]:
import re

# Pre-processing
remove_symbols = re.compile('[^0-9A-Za-z ]') 

def clean_text(input):
  input = remove_symbols.sub(' ', str(input)) 
  # remove  empty spaces
  re.sub('\s+', '' ,input)
  return input

df_balanced['tweet'] = df_balanced['tweet'].apply(clean_text)
df_balanced.head(10)

,tweet,sarcastic
0,I always think going braless is a good idea un...,0
1,life is so much better with a heating blanket,0
2,Sometimes I just go through my phone and look ...,0
3,was not back in the states for even 5 minutes ...,0
4,in desperate need of and I can NOT stress thi...,0
5,I ve said it before and I ll say it again but ...,0
6,I couldn t have imagined how much fun I would ...,0
7,woke up to my dog sneezing on my face How s y...,0
8,Does the salad offset the beer s,0
9,Why do I have a doctorate and miss the restaur...,0


In [10]:
df_test['text'] = df_test['text'].apply(clean_text)

In [11]:
!pip install tensorflow_text
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")

# Bert layers
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)

# Neural network layers
l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
l = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(l)

# Use inputs and outputs to construct a final model
model = tf.keras.Model(inputs=[text_input], outputs = [l])

METRICS = [
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall')
]

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=METRICS)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 96.3 MB/s eta 0:00:00


Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


In [17]:
model.fit(df_balanced['tweet'], df_balanced['sarcastic'], epochs=5)

Epoch 1/5
217/217 [==============================] - 78s 359ms/step - loss: 0.6817 - accuracy: 0.5686 - precision: 0.5668 - recall: 0.5822
Epoch 2/5
217/217 [==============================] - 75s 344ms/step - loss: 0.6738 - accuracy: 0.5802 - precision: 0.5790 - recall: 0.5874
Epoch 3/5
217/217 [==============================] - 75s 348ms/step - loss: 0.6707 - accuracy: 0.5913 - precision: 0.5884 - recall: 0.6073
Epoch 4/5
217/217 [==============================] - 75s 345ms/step - loss: 0.6712 - accuracy: 0.5924 - precision: 0.5904 - recall: 0.6035
Epoch 5/5
217/217 [==============================] - 75s 345ms/step - loss: 0.6675 - accuracy: 0.5918 - precision: 0.5883 - recall: 0.6116


In [18]:
y_predicted = model.predict(df_test['text'])
y_predicted = y_predicted.flatten()
y_predicted

import numpy as np

y_predicted = np.where(y_predicted > 0.5, 1, 0)
y_predicted

from sklearn.metrics import confusion_matrix, classification_report

cm = confusion_matrix(df_test['sarcastic'], y_predicted)
cm 

44/44 [==============================] - 15s 341ms/step


array([[962, 238],
       [147,  53]])

In [19]:
print(classification_report(df_test['sarcastic'], y_predicted))

              precision    recall  f1-score   support

           0       0.87      0.80      0.83      1200
           1       0.18      0.27      0.22       200

    accuracy                           0.73      1400
   macro avg       0.52      0.53      0.52      1400
weighted avg       0.77      0.72      0.75      1400



In [20]:
from sklearn.metrics import f1_score
f1 = f1_score(df_test['sarcastic'], y_predicted,average='macro')
f1

0.524573549530351

In [16]:
df_test['sarcastic'].value_counts()

0    1200
1     200
Name: sarcastic, dtype: int64